In [ ]:
import chromedriver_autoinstaller
from selenium import webdriver
import pandas as pd
import re
import math
import time
import os, sys
import requests
import numpy as np
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver import ActionChains

In [ ]:
### 크롬드라이버 자동 설치 및 업데이트 후 실행 함수

def open_chromedriver():
    
    #현재 작업 디렉토리에 크롬드라이버 설치 또는 업데이트
    chrome = chromedriver_autoinstaller.install(os.getcwd()) 
    options = webdriver.ChromeOptions()
    
    ### 크롬 드라이버 옵션 헤드리스 추가(크롬 창 숨기기), options.headless =True의 주석처리하면 작업 중인 크롬창 확인 가능
    options.headless = True 
    
    
    ### 옵션 추가한 채로 크롬 드라이버 실행
    driver = webdriver.Chrome(options=options) 
    return(driver)


In [ ]:
### 대 카테고리 url 수집 함수

def get_category_url():
    
    ###페이지에서 전체상품의 xpath 값 가져와서 해당 위치 찾고 클릭
    driver.find_element_by_xpath('//*[@id="wrap"]/header/ul/li[1]/em').click() 

    title = []
    link = []
    
    ### 전체상품엑서 대카테고리 이름과 url 가져오기
    for i in driver.find_elements_by_xpath('//*[@id="menuCate"]/li[*]'): 
        title.append(i.text)
        link.append(i.find_element_by_tag_name('a').get_attribute('href'))
    df = {'title':title, 'link':link}
    df = pd.DataFrame(df)

    text_1 = []
    link_1 = []
    up = []
    
    
    for i in range(0,len(df)):
        
        ###대카테고리로 이동
        driver.get(df['link'][i]) 

        time.sleep(0.5)
        
        ###대카테고리에서 하위 카테고리 이름과 링크 수집
        for j in driver.find_elements_by_xpath('//*[@id="sub_ctg"]/li[*]/a'): 
            text_1.append(j.text)
            link_1.append(j.get_attribute('onclick'))
        
        up.extend([df['title'][i]]*len(driver.find_elements_by_xpath('//*[@id="sub_ctg"]/li[*]/a')))

    df = pd.DataFrame({'대카테고리':up,'서브카테고리':text_1,'서브카테고리 링크':link_1})
    return(df)

In [ ]:
### 페이지 내리는 함수

def page_scroll_down():
    
    ### 페이지 두 번 새로고침(페이지가 느려서 렉 걸리는 경우를 방지하기 위해)
    driver.refresh()
    driver.refresh()
    
    time.sleep(0.8)
    while True:
        
        ### 해당 카테고리의 상품 수를 product_num 이라는 변수로 선언
        product_num = int(driver.find_element_by_xpath('//*[@id="dpItemListWrap"]/h3/span').text)
        
        ### 페이지 끝까지 내려서 상품 더 확인하고 페이지를 끝까지 내려도 상품 수가 변하지 않으면
        ### 마지막 페이지로 인식하고 페이지 내리는 작업 종료
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(0.6)
        
        print(len(driver.find_elements_by_xpath('//*[@id="dpItemList"]/li[*]')),'/',product_num)

        if product_num ==len(driver.find_elements_by_xpath('//*[@id="dpItemList"]/li[*]')):
            break
        if product_num < len(driver.find_elements_by_xpath('//*[@id="dpItemList"]/li[*]')):
            driver.refresh()
            driver.refresh()
            driver.refresh()
            time.sleep(0.8)
    return(product_num)

In [ ]:
### 태그 수집하는 함수

def get_tag():
    tag = []
    for i in range(1,product_num+1):
        temp=[]
        
        ###태그의 xpath값 찾아서 해당 데이터 가져온 후 값이 없으면 NULL 있으면 해당 텍스트 가져오기
        for j in driver.find_elements_by_xpath('//*[@id="dpItemList"]/li[%d]/div/div[1]/a/div[2]/div[*]/span' %i) :
            if len(j.text)==0:
                temp.append('NULL')
            else:
                temp.append(j.text)
        tag.append(temp)
    return(tag)

In [ ]:
### 상품 링크 가져오는 함수

def get_product_link():
    product_onclick=[]
    
    ###상품들의 링크(onclick 값) xpath 값 가져와서 해당 데이터 가져오기
    for i in driver.find_elements_by_xpath('//*[@id="dpItemList"]/li[*]/div/div[1]/a'):
        product_onclick.append(i.get_attribute('onclick'))
    return(product_onclick)

In [ ]:
### 상품 이름 가져오는 함수
def get_prd_name():
    
    ### 상품 이름에 해당하는 class값 가져와서 데이터 수집
    name = driver.find_element_by_class_name('prd_name').text
    return(name)

In [ ]:
### 상품 가격 가져오는 함수
def get_prd_price():
    
    ### 상품 가격에 해당하는 xpath값 가져와서 해당 부분 텍스트 가져온 후 '원'은 삭제해서 숫자만 남기기
    price=driver.find_element_by_xpath('//*[@id="afterCost"]').text
    price = re.sub('원','',price)
    return(price)

In [ ]:
### 상품 좋아요 수 가져오는 함수
def get_prd_like():
    
    ###상품 좋아요에 해당하는 부분의 class 가져와서 텍스트 값 가져오기
    like = driver.find_element_by_class_name('prd_like').text
    return(like)

In [ ]:
### 배송 관련 정보 가져오는 함수
def get_prd_deliver():
    
    ### 배송 관련 정보에 해당하는 class값 가져와서 해당 텍스트 수집, 없다면 NULL값 추가
    try:
        deliver = driver.find_element_by_class_name('prd_deliver').text
    except:
        deliver="NULL"
    return(deliver)

In [ ]:
## 상품 상세 정보 가져오는 함수
def get_prd_spec():
    spec_col = []
    spec_data = []
    
    ### 상품 상세정보에서 tag 값이 'dt' 부분이 열 이름이므로 해당 부분 가져오기
    for i in driver.find_element_by_class_name('prd_spec').find_elements_by_tag_name('dt'):
        spec_col.append(i.text)
        
    ### 상품 상세정보에서 tag 값이 'dd' 부분이 데이터에 해당함으로 해당 부분 가져오기
    for i in driver.find_element_by_class_name('prd_spec').find_elements_by_tag_name('dd'):
        spec_data.append(i.text)
        
    ### 열이름에 해당하는 부분과 데이터에 해당하는 부분 합쳐서 데이터 프레임으로 만들기
    spec = pd.DataFrame([spec_data], columns=spec_col)
    return(spec)

In [ ]:
### 상품 혜택 정보 가져오는 함수
def get_prd_flag():
    
    ### 상품 혜택 정보에 해당하는 class값 가져와서 text 데이터 추출, 없다면 NULL값 추가
    try:
        flag=driver.find_element_by_class_name('flag').text
    except:
        flag = 'NULL'
    return(flag)

In [ ]:
### 리뷰 숫자 가져오는 함수

def get_review_num():
    
    ###구매 후기에 해당하는 xpath값 가져와서 텍스트 추출
    num =driver.find_element_by_xpath('//*[@id="reviewTitle"]/h3/span').text
    return(num)

In [ ]:
### 전체 구매후기 보기 위치로 페이지 스크롤 밑 전체 구매 후기 보기 클릭하는 함수
def review_all() :
    
    ### 상품 문의 xpath 값 가져와서 label 변수 선언하고 해당 위치로 페이지 이동
    ### 크롬 드라이버를 사용하여 클릭할 때 해당 위치가 화면에 보이지 않으면 클릭되지 않는 에러를 없애기 위해
    action = ActionChains(driver)
    label = driver.find_element_by_xpath('//*[@id="qnaTitle"]/h3')
    action.move_to_element(label).perform()
    
    ### 전체 구매 후기 버튼 클릭
    driver.find_element_by_xpath('//*[@id="buyAfterListCount"]').click() 
    time.sleep(1.5)

In [ ]:
### 리뷰가 전체가 보일 수 있도록 스크롤 내리는 함수

def review_page_scroll():
    while True:
        
        ### num에 현재 리뷰 수를 선언하고 페이지를 내렸을 때 현재와 리뷰 수가 같다면 페이지 내리기 중단
        num = len(driver.find_elements_by_xpath('//*[@id="container"]/article/div/ul/li[*]'))
        print(num)
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(0.3)

        if num ==len(driver.find_elements_by_xpath('//*[@id="container"]/article/div/ul/li[*]')):
            break

In [ ]:
### 리뷰 아이디 가져오기

def get_review_id():
    review_id = []
    
    ### 리뷰 아이디의 class 값으로 데이터 가져오고 데이터가 없다면 NULL 추가
    for i in driver.find_elements_by_class_name('name'):
        try:
            text=re.split('\n',i.text)[0]
            if len(text)!=0:
                review_id.append(text)
            else:
                review_id.append('NULL')
        except:
            review_id.append('NULL')
    return(review_id)

In [ ]:
### 리뷰 날짜 가져오기

def get_review_date():
    date=[]
    
    ### 리뷰 날짜의 class 값으로 데이터 가져오고 데이터가 없다면 NULL 추가
    for i in driver.find_elements_by_class_name('date'):
        try:
            text= i.text
            if len(text)!=0:
                date.append(text)
            else:
                date.append('NULL')
        except:
            date.append("NULL")
        
    return(date)

In [ ]:
### 리뷰 만족 가져오기

def get_review_satisfied():
    review_satisfied = []
    
    ### 리뷰의 만족도에 해당하는 class 값 가져오고 데이터가 없다면 NULL 추가
    for i in driver.find_elements_by_class_name('name'):
        try:
            text = re.split('\n',i.text)[1]
            if len(text)!=0:
                review_satisfied.append(text)
        except:
            review_satisfied.append('NULL')
    return(review_satisfied)

In [ ]:
### 리뷰 내용 가져오는 함수

def get_review_contents():
    review_contents=[]
    
    ### 리뷰 내용의 class 값 가져와서 해당 부분의 데이터 추출
    for i in driver.find_elements_by_class_name('txt'):
        review_contents.append(i.text)
    return(review_contents)

In [ ]:
### 원격 크롬 창 실행 후 driver에 변수 선언

driver = open_chromedriver() 

In [ ]:
### 헬로네이쳐 페이지로 이동

driver.get('https://www.hellonature.co.kr/')

In [ ]:
### 팝업창 떠있으면 닫기 누르고 없으면 취소

try:
    driver.find_element_by_xpath('//*[@id="mainPop3"]/div/a').click()
except:
    None

In [ ]:
df = get_category_url()

In [ ]:
num = 0
tag=[]
product_link=[]
up=[]

for i in df['서브카테고리 링크']:
    num=num+1
    
    print(num,'/',len(df))
    
    driver.refresh()
    
    time.sleep(1)
    
    driver.execute_script(i)
    
    time.sleep(1)
    
    product_num = page_scroll_down()
    
    tag.extend(get_tag())
    product_link.extend(get_product_link())
    up.extend([i]*int(product_num))
    
    print(len(tag),len(up),len(product_link))

In [ ]:
df1=pd.DataFrame({'태그':tag,'서브카테고리 링크':up,'상품 링크':product_link})

In [ ]:
df2=pd.merge(df, df1,how='right')

In [ ]:
df2_1=df2[df2['서브카테고리']=='전체']

In [ ]:
df2_2 = df2[df2['서브카테고리']!='전체']

In [ ]:
prd_name =[]
prd_price=[]
prd_like=[]
prd_deliver=[]
prd_spec = pd.DataFrame()
prd_flag =[]
review_num = []
review_id =[]
review_date =[]
review_satisfied=[]
review_contents=[]
matching_key1= []
matching_key2 = []
num=0
for i in df2_1['상품 링크']:
    num=num+1
    print(num,'/',len(df2_1))
    driver.execute_script(i)
    
    time.sleep(0.5)

    prd_name.append(get_prd_name())
    prd_price.append(get_prd_price())
    prd_like.append(get_prd_like())
    prd_deliver.append(get_prd_deliver())
    prd_spec= pd.concat([prd_spec,get_prd_spec()],axis=0)
    prd_flag.append(get_prd_flag())
    temp=get_review_num()
    review_num.append(temp)
    matching_key1.append(i)
    
    if int(re.sub('[^0-9]','',temp))==0:
        review_id.append('')
        review_date.append('')
        review_satisfied.append('')
        review_contents.append('')
        matching_key2.append(i)
        continue
        
    review_all()

        
    review_page_scroll()

    review_id.extend(get_review_id())
    review_date.extend(get_review_date())
    review_satisfied.extend(get_review_satisfied())
    review_contents.extend(get_review_contents())
    
    matching_key2.extend([i]*int(re.sub('[^0-9]','',temp)))
    
    driver.back()
    driver.refresh()
    
    time.sleep(0.5)

    if num%50==0:
        driver.quit()

        driver = open_chromedriver()

        driver.get('https://www.hellonature.co.kr/fdp001.do?goTo=dpItemList&gubunFlag=D&pageSize=10&filterYn=N&ctgrCd=010000&dpItemListCntYn=Y&page=1')

        time.sleep(2)

In [ ]:
prd_flag=['' if value=='답변완료' else value for value in prd_flag]

In [ ]:
prd_flag=['NULL' if value=='' else value for value in prd_flag]

In [ ]:
frame1 = pd.DataFrame({'prd_name':prd_name,'prd_price':prd_price,'prd_like':prd_like,'prd_deliver':prd_deliver,'prd_flag':prd_flag,
'review_num':review_num,'matching_key1':matching_key1})

In [ ]:
prd_spec=prd_spec.reset_index().drop(['index'],axis=1)

In [ ]:
frame1 = pd.concat([frame1, prd_spec], axis=1)

In [ ]:
frame2=pd.DataFrame({'review_id':review_id,'review_date':review_date,'review_satisfied':review_satisfied,'revie_contents':review_contents,'matching_key2':matching_key2})

In [ ]:
frame3=pd.merge(frame1,frame2,how='right',left_on = 'matching_key1',right_on='matching_key2')

In [ ]:
frame3=frame3.drop(['matching_key2'],axis=1)

In [ ]:
frame = pd.merge(df2_2, frame3, how='right', left_on = '상품 링크',right_on='matching_key1')

In [ ]:
writer = pd.ExcelWriter('hellonatrue.xlsx', engine='xlsxwriter', options={'strings_to_urls': True})
frame.to_excel(writer)
writer.close()